In [1]:
import numpy as np
from hmm_utils import HMM
from attackers.smoothing_state_attacker import ss_attacker
from params import *
from solvers.mc_ennumeration import MC_enumeration
from solvers.simulated_annealing import simulated_annealing

from scipy.special import softmax

In [2]:
priors     = np.array([0.5,0.3,0.2])
transition = np.array([[0.85, 0.05,0.1],
                       [0.05, 0.9,0.05],
                        [1/2, 1/4, 1/4]])

emission   = np.array([[0.699, 0.05, 0.1, 0.05, 0.1, 0.001],
                    [0.001, 0.1, 0.1, 0.299, 0.3, 0.2],
                       [0.1, 0.2, 0.1, 0.2, 0.1, 0.3]])

rho_probs = np.ones(6)

x_vector = np.array([ [5], [3], [4], [3], [5]])

In [3]:
attacker = ss_attacker(priors, transition, emission, rho_probs,
         x_vector, w1 = 1.0 ,w2 = 0.0 , t = 3, state= 0, c=1, k_value=100000)

Z_set = attacker.generate_attacks()

# Full ennumeration

In [4]:
z_star, z_vec = MC_enumeration(attacker, 10)

Percentage completed: 0.0
Percentage completed: 12.86
Percentage completed: 25.72
Percentage completed: 38.58
Percentage completed: 51.44
Percentage completed: 64.3
Percentage completed: 77.16
Percentage completed: 90.02


In [5]:
np.argsort(-z_vec)[:10]

array([   0,    2, 2592, 5184,    4, 2594, 5186,    1, 1296, 3888])

In [7]:
exp_util(10, x_vector, hmm, Z_set[2], 1000)

6.988974659911887

In [16]:
attacker.expected_utility(Z_set[0], 10)

0.989442499362112

In [21]:
Z_set[0]

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

# Simulated annealing

In [4]:
z_star = simulated_annealing(attacker, 200, verbose=True)

Percentage completed: 0.0
Current state [[0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
Percentage completed: 25.0
Current state [[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
Percentage completed: 50.0
Current state [[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
Percentage completed: 75.0
Current state [[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [5]:
z_star

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

# Gibbs - APS

In [249]:
def update_probs_metropolis(H, hmm, hmms_old, rho_matrix_old, z_matrix, k_value, old_value):
    
    transtion_mat_list = hmm.sample_mat(hmm.transmat_, n = H, k = k_value)
    emission_mat_list = hmm.sample_mat(hmm.emissionprob_, n = H, k = k_value)
    priors_vec_list = hmm.sample_mat(np.array([list(hmm.startprob_)]), n = H, k = k_value)
    rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)
    
    hmms = []
    rho_matrix_samples = []
    
    value = 0
    
    for h in range(H):
        
        hmm_n = HMM(hmm.n_components, hmm.n_obs)
        hmm_n.startprob_ = priors_vec_list[h].reshape(-1)
        hmm_n.transmat_ = transtion_mat_list[h]
        hmm_n.emissionprob_ = emission_mat_list[h]
        rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)
        
        hmms.append(hmm_n)
        rho_matrix_samples.append(rho_matrix)
        
        y_t = hmm_n.attack_X(X = x_vector, z_matrix = z_matrix, 
                             rho_matrix = rho_matrix).astype('int')
        
        value += np.log( 
            attacker.utility(hmm = hmm_n, z_matrix = z_matrix, 
                             x_obs_vector = x_vector, y_t = y_t) + 6 )
        
    value /= H

    prob =  np.exp(H*value - H*old_value)

    
    if np.random.uniform() < prob:
        
        print("Accept!")
        
        return hmms, rho_matrix_samples, value
    
    else:
        
        return hmms_old, rho_matrix_old, old_value
    
    
def update_z(z_init, hmms, rho_matrix_samples, idx):
    
    Z_candidates = np.apply_along_axis( 
        lambda x: return_mat(x, z_init, idx), 1, np.eye(6))

    energies = np.zeros( len(Z_candidates) )
    
    H = len(hmms)
    print(H)

    for i, z in enumerate(Z_candidates):
        
        for h in range(H):
            
            y_t = hmms[h].attack_X(X = x_vector,
                                   z_matrix = z, rho_matrix = rho_matrix_samples[h]).astype('int')
            
            energies[i] += np.log (
                attacker.utility(hmm = hmms[h], 
                                 z_matrix = z, x_obs_vector = x_vector, y_t = y_t) + 6  )
            

    p = softmax(energies)
    print(p)
    
    candidate_idx = np.random.choice( 
        np.arange(len(Z_candidates) ), p=p )
    
    return Z_candidates[candidate_idx]

        
    

In [252]:
z_init = Z_set[ np.random.choice(Z_set.shape[0]) ]

hh = 15000
k_value = 1000
##
transtion_mat_list = hmm.sample_mat(hmm.transmat_, n = hh, k = k_value)
emission_mat_list = hmm.sample_mat(hmm.emissionprob_, n = hh, k = k_value)
priors_vec_list = hmm.sample_mat(np.array([list(hmm.startprob_)]), n = hh, k = k_value)
rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)

hmms = []
rho_matrix_samples = []

value = 0
for temp in range(hh):

    hmm_n = HMM(hmm.n_components, hmm.n_obs)
    hmm_n.startprob_ = priors_vec_list[temp].reshape(-1)
    hmm_n.transmat_ = transtion_mat_list[temp]
    hmm_n.emissionprob_ = emission_mat_list[temp]
    rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)

    hmms.append(hmm_n)
    rho_matrix_samples.append(rho_matrix)

    y_t = hmm_n.attack_X(X = x_vector, z_matrix = z_init, rho_matrix = rho_matrix).astype('int')
    value += np.log( 
        attacker.utility(hmm = hmm_n, z_matrix = z_init, x_obs_vector = x_vector, y_t = y_t) + 6 )
    

value /= hh
value   


1.8118138218839588

In [253]:
for temp in np.arange(15000, 20000, 250):
 
    #for times in range(5):
        
    # Update z
    for idx in range(5):
        z_init = update_z(z_init, hmms, rho_matrix_samples, idx)
        print(z_init)

    # Update value with new z
    value = 0
    for n in range(len(hmms)):


        y_t = hmms[n].attack_X(X = x_vector, z_matrix = z_init, 
                               rho_matrix = rho_matrix_samples[n]).astype('int')
        value += np.log( 
            attacker.utility(hmm = hmms[n], z_matrix = z_init, 
                             x_obs_vector = x_vector, y_t = y_t) + 6 )


    value /= len(hmms)



    hmms, rho_matrix_samples, value = update_probs_metropolis(temp, hmm, hmms, 
                                                        rho_matrix_samples, z_init, k_value, value)




15000
[1.00000000e+000 3.56626593e-210 1.66549428e-185 2.10953036e-255
 5.09770397e-245 9.14638465e-255]
[[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
15000
[1.00000000e+000 1.79430630e-232 1.96678654e-139 2.67753528e-299
 3.65568586e-227 0.00000000e+000]
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
15000
[1.00000000e+000 0.00000000e+000 3.67418672e-184 0.00000000e+000
 2.28840039e-283 0.00000000e+000]
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
15000
[9.99999303e-01 2.16709357e-14 6.97098853e-07 1.54890967e-21
 1.11570299e-12 1.38811111e-38]
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
15000
[0.3726974  0.1355502  0.23695337 0.07932031 0.1530084  0.02247032]
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0.

KeyboardInterrupt: 

# Full APS

In [197]:
def propose_z(z):
    z_new = z.copy()
    idx = np.random.choice(np.arange(z.shape[0]))
    candidates  = np.diag( np.ones(z.shape[1]) )
    candidates  = candidates[~np.all(candidates == z[idx], axis=1)]

    z_new[idx] = candidates[ np.random.choice(np.arange(candidates.shape[0])) ]
    return z_new

def update_z(H, hmm, z_new, z_old, old_value, k_value):
    
    transtion_mat_list = hmm.sample_mat(hmm.transmat_, n = H, k = k_value)
    emission_mat_list = hmm.sample_mat(hmm.emissionprob_, n = H, k = k_value)
    priors_vec_list = hmm.sample_mat(np.array([list(hmm.startprob_)]), n = H, k = k_value)
    
    hmms = []
    rho_matrix_samples = []
    
    value = 0
    
    for h in range(H):
        
        hmm_n = HMM(hmm.n_components, hmm.n_obs)
        hmm_n.startprob_ = priors_vec_list[h].reshape(-1)
        hmm_n.transmat_ = transtion_mat_list[h]
        hmm_n.emissionprob_ = emission_mat_list[h]
        rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)
        
        hmms.append(hmm_n)
        rho_matrix_samples.append(rho_matrix)
        
        y_t = hmm_n.attack_X(X = x_vector, z_matrix = z_new, 
                             rho_matrix = rho_matrix).astype('int')
        
        value += np.log( 
            attacker.utility(hmm = hmm_n, z_matrix = z_new, 
                             x_obs_vector = x_vector, y_t = y_t) + 6 )
        
    value /= H

    prob =  np.exp(H*value - H*old_value)

    
    if np.random.uniform() < prob:
        
        print("Accept!")
        print(prob)
        print(z_new)
        print(z_old)
        
        return z_new, value
    
    else:
        
        return z_old, old_value

In [211]:
z_init = Z_set[ np.random.choice(Z_set.shape[0]) ]

z_init =np.array([[0., 1., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.]])


hh = 15000
k_value = 1000000
##
transtion_mat_list = hmm.sample_mat(hmm.transmat_, n = hh, k = k_value)
emission_mat_list = hmm.sample_mat(hmm.emissionprob_, n = hh, k = k_value)
priors_vec_list = hmm.sample_mat(np.array([list(hmm.startprob_)]), n = hh, k = k_value)
rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)

hmms = []
rho_matrix_samples = []

value = 0
for temp in range(hh):

    hmm_n = HMM(hmm.n_components, hmm.n_obs)
    hmm_n.startprob_ = priors_vec_list[temp].reshape(-1)
    hmm_n.transmat_ = transtion_mat_list[temp]
    hmm_n.emissionprob_ = emission_mat_list[temp]
    rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)

    hmms.append(hmm_n)
    rho_matrix_samples.append(rho_matrix)

    y_t = hmm_n.attack_X(X = x_vector, z_matrix = z_init, rho_matrix = rho_matrix).astype('int')
    value += np.log( 
        attacker.utility(hmm = hmm_n, z_matrix = z_init, x_obs_vector = x_vector, y_t = y_t) + 6 )
    

value /= hh
value   


1.9443352596947014

In [212]:
for temp in np.arange(15000, 20000, 250):

    z_new = propose_z(z_init)
    z_init, value = update_z(temp, hmm, z_new, z_init, value, k_value)
    
    print(temp)
    print(z_init)

   


15000
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
15250
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
15500
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
Accept!
1.524466408672834
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
15750
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
16000
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
16250
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
16500
[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]


KeyboardInterrupt: 

# Trash

In [305]:
def update_probs(H, hmm, k_value):
    
    transtion_mat_list = hmm.sample_mat(hmm.transmat_, n = H, k = k_value)
    emission_mat_list = hmm.sample_mat(hmm.emissionprob_, n = H, k = k_value)
    priors_vec_list = hmm.sample_mat(np.array([list(hmm.startprob_)]), n = H, k = k_value)
    rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)
    
    hmms = []
    rho_matrix_samples = []
    
    for h in range(H):
        
        hmm_n = HMM(hmm.n_components, hmm.n_obs)
        hmm_n.startprob_ = priors_vec_list[h].reshape(-1)
        hmm_n.transmat_ = transtion_mat_list[h]
        hmm_n.emissionprob_ = emission_mat_list[h]
        rho_matrix = hmm.sample_rho(5, np.ones(6) * 1.0)
        
        hmms.append(hmm_n)
        rho_matrix_samples.append(rho_matrix)
    
    return hmms, rho_matrix_samples

def update_z(z_init, hmms, rho_matrix_samples, idx):
    
    Z_candidates = np.apply_along_axis( 
        lambda x: return_mat(x, z_init, idx), 1, np.eye(6))

    energies = np.zeros( len(Z_candidates) )
    
    H = len(hmms)

    for i, z in enumerate(Z_candidates):
        
        for h in range(H):
            
            y_t = hmms[h].attack_X(X = x_vector,
                                   z_matrix = z, rho_matrix = rho_matrix_samples[h]).astype('int')
            
            energies[i] += np.log (
                attacker.utility(hmm = hmms[h], z_matrix = z, x_obs_vector = x_vector, y_t = y_t) )
        

    p = np.exp(energies)/np.sum(np.exp(energies))
    
    candidate_idx = np.random.choice( 
        np.arange(len(Z_candidates) ), p=p )
    
    return Z_candidates[candidate_idx]


In [ ]:
z_init = Z_set[ np.random.choice(Z_set.shape[0]) ]

h_grid = [100, 1000, 10000]

for h in h_grid:
    
    for times in range(5):
        hmm_init, rho_matrix_init = update_probs(h, hmm, 1000000)
        
        for idx in range(5):
            z_init = update_z(z_init, hmm_init, rho_matrix_init, idx)
        print(z_init)
    



In [170]:
z_init

array([[0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [175]:
z_init =np.array([[1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.]])


for times in range(50):
    hmm_init, rho_matrix_init = update_probs(20, hmm, 1000000)
    for i in range(5):
        idx = np.random.choice(5)
        z_init = update_z(z_init, hmm_init, rho_matrix_init, idx)
    print(z_init)

    

[[0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]]
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]]
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1.

In [299]:
z_init =np.array([[0., 0., 1., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.],
                 [1., 0., 0., 0., 0., 0.]])

hmm_init, rho_matrix_init = update_probs(10000, hmm, 10000000)

update_z(z_init, hmm_init, rho_matrix_init, 0)


[8.75357835e-01 4.29378681e-03 9.06900356e-02 1.27118632e-03
 2.83704881e-02 1.66680554e-05]


array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])